## **Predikcia Platov v AI Sektore Pomocou Strojového učenia**



**Abstrakt:**  
V tomto projekte som vytvorila a porovnala regresné modely (Lineárna regresia, Random Forest) na predikciu ročných platov v sektore umelej inteligencie. Notebook obsahuje kompletné fázy modelovania, interpretácie výsledkov pomocou SHAP a prototyp nasadenia modelu v Gradio.

---

# Obsah

1. Úvod a Kontext Projektu  
2. Načítanie dát a Import knižníc  
3. Jednoduchá lineárna regresia  
4. Viacnásobná lineárna regresia  
5. Rozšírená viacnásobná regresia  
6. Random Forest model  
7. Interpretácia modelov – SHAP  
8. Porovnanie modelov  
9. Gradio aplikácia  
10. Zhrnutie, záver a odporúčania  



## 1. Úvod a Kontext Projektu

Tento projekt sa zameriava na vytvorenie, vyhodnotenie a porovnanie regresných modelov strojového učenia (konkrétne **Lineárna Regresia** a **Random Forest**) pre **predikciu ročných platov v sektore umelej inteligencie**. Cieľom je nielen dosiahnuť čo najpresnejšie predpovede, ale aj porozumieť kľúčovým faktorom, ktoré ovplyvňujú výšku platu.

Notebook sa detailne venuje fázam **modelovania, predikcie a interpretácie výsledkov**, ako aj **prototypovaniu nasadenia modelu** pomocou knižnice Gradio.

Vychádza z komplexnej analýzy trhu práce v AI sektore, ktorá je detailne popísaná a vizualizovaná v súvisiacom notebooku [ Global AI job market.ipynb](AI%20job%20market.ipynb).

---

##  2. Načítanie dát a import potrebných knižníc
Prvým krokom je import nevyhnutných knižníc a načítanie  a dát zo súboru `ai_job.csv` a vytvorenie dataframe. Zobrazenie základných informácií o dataframe.


In [ ]:
# Základné knižnice
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Modelovanie
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Vyhodnotenie
from sklearn.metrics import mean_squared_error, r2_score

# Interpretácia
import shap

# Načítanie dát
df = pd.read_csv("ai_job.csv")
df.info()

## 3. Jednoduchá Lineárna Regresia
Začíname s najjednoduchším modelom, kde sa pokúsime predpovedať plat (`salary_usd`) iba na základe jednej premennej – pomeru práce na diaľku (`remote_ratio`). Očakávame, že tento model nebude veľmi presný, ale slúži ako dobrý základ pre porovnanie.

In [ ]:
# Príprava dát pre jednoduchú lineárnu regresiu
X_simple = df[['remote_ratio']]
y_simple = df['salary_usd']

# Rozdelenie dát
X_train_simple, X_test_simple, y_train_simple, y_test_simple = train_test_split(X_simple, y_simple, test_size=0.2, random_state=42)

# Trénovanie modelu
model_simple = LinearRegression()
model_simple.fit(X_train_simple, y_train_simple)

# Predikcia a vyhodnotenie
y_pred_simple = model_simple.predict(X_test_simple)
r2_simple = r2_score(y_test_simple, y_pred_simple)
mae_simple = mean_absolute_error(y_test_simple, y_pred_simple)
rmse_simple = np.sqrt(mean_squared_error(y_test_simple, y_pred_simple))

print("--- Výsledky Jednoduchej Lineárnej Regresie ---")
print(f"R-squared (R²): {r2_simple:.4f}")
print(f"Mean Absolute Error (MAE): {mae_simple:,.2f} USD")
print(f"Root Mean Squared Error (RMSE): {rmse_simple:,.2f} USD")

## 4. Viacnásobná Lineárna Regresia
Teraz model rozšírime o ďalšie relevantné kategorické premenné: `experience_level`, `company_size` a `employment_type`.  Model jednoduchej lineárnej regresie nebol dostatočne presný, takže sme pridali ďalšie premnné.Tieto premenné musíme najprv transformovať pomocou One-Hot Encodingu.

In [ ]:
# Príprava dát pre viacnásobnú lineárnu regresiu
feature_cols_multi = ['experience_level', 'company_size', 'remote_ratio', 'employment_type']
X_multi = df[feature_cols_multi]
y_multi = df['salary_usd']

# One-Hot Encoding
X_multi_encoded = pd.get_dummies(X_multi, drop_first=True)

# Rozdelenie dát
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X_multi_encoded, y_multi, test_size=0.2, random_state=42)

# Trénovanie modelu
model_multi = LinearRegression()
model_multi.fit(X_train_multi, y_train_multi)

# Predikcia a vyhodnotenie
y_pred_multi = model_multi.predict(X_test_multi)
r2_multi = r2_score(y_test_multi, y_pred_multi)
mae_multi = mean_absolute_error(y_test_multi, y_pred_multi)
rmse_multi = np.sqrt(mean_squared_error(y_test_multi, y_pred_multi))

print("\n--- Výsledky Viacnásobnej Lineárnej Regresie ---")
print(f"R-squared (R²): {r2_multi:.4f}")
print(f"Mean Absolute Error (MAE): {mae_multi:,.2f} USD")
print(f"Root Mean Squared Error (RMSE): {rmse_multi:,.2f} USD")

## 5. Rozšírená Viacnásobná Lineárna Regresia
Pre ešte lepšiu presnosť pridáme do modelu kľúčové premenné `job_title` a `company_location`. Tým sa výrazne zvýši počet features po One-Hot Encodingu.

In [ ]:
# Príprava dát pre rozšírenú viacnásobnú lineárnu regresiu
feature_cols_expanded = ['experience_level', 'company_size', 'remote_ratio', 'employment_type', 'job_title', 'company_location']
X_expanded = df[feature_cols_expanded]
y_expanded = df['salary_usd']

# One-Hot Encoding
X_expanded_encoded = pd.get_dummies(X_expanded, drop_first=True)

# Rozdelenie dát
X_train_expanded, X_test_expanded, y_train_expanded, y_test_expanded = train_test_split(X_expanded_encoded, y_expanded, test_size=0.2, random_state=42)

# Trénovanie modelu
model_expanded = LinearRegression()
model_expanded.fit(X_train_expanded, y_train_expanded)

# Predikcia a vyhodnotenie
y_pred_expanded = model_expanded.predict(X_test_expanded)
r2_expanded = r2_score(y_test_expanded, y_pred_expanded)
mae_expanded = mean_absolute_error(y_test_expanded, y_pred_expanded)
rmse_expanded = np.sqrt(mean_squared_error(y_test_expanded, y_pred_expanded))

print("\n--- Výsledky Rozšírenej Viacnásobnej Lineárnej Regresie ---")
print(f"R-squared (R²): {r2_expanded:.4f}")
print(f"Mean Absolute Error (MAE): {mae_expanded:,.2f} USD")
print(f"Root Mean Squared Error (RMSE): {rmse_expanded:,.2f} USD")

## 6. Random Forest Regressor
Ako alternatívu k lineárnym modelom vyskúšame `RandomForestRegressor`. Tento model je založený na stromoch a dokáže lepšie zachytiť nelineárne vzťahy v dátach. Použijeme rovnakú sadu rozšírených features.

In [ ]:
# Trénovanie modelu Random Forest
# n_jobs=-1 využije všetky dostupné jadrá procesora na zrýchlenie trénovania
model_rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_rf.fit(X_train_expanded, y_train_expanded)

# Predikcia a vyhodnotenie
y_pred_rf = model_rf.predict(X_test_expanded)
r2_rf = r2_score(y_test_expanded, y_pred_rf)
mae_rf = mean_absolute_error(y_test_expanded, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test_expanded, y_pred_rf))

print("\n--- Výsledky Random Forest Regressora ---")
print(f"R-squared (R²): {r2_rf:.4f}")
print(f"Mean Absolute Error (MAE): {mae_rf:,.2f} USD")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:,.2f} USD")

# 7. SHAP Interpretácia modelu

In [ ]:
import shap

# Vytvoríme vzorku 10 riadkov zo testovacích dát
sample = X_test_expanded.sample(10, random_state=42)

# Vytvoríme explainer (ak ešte nemáš)
explainer = shap.TreeExplainer(model_rf)

# Vypočítame SHAP hodnoty pre vzorku
shap_values = explainer.shap_values(sample)

# Zobrazíme summary plot pre túto vzorku
shap.summary_plot(shap_values, sample)


## 8. Porovnanie Modelov a Výber Najlepšieho
Teraz zhrnieme výsledky všetkých štyroch modelov do prehľadnej tabuľky a grafu, aby sme jednoznačne určili, ktorý model je najvýkonnejší.

In [ ]:
# Vytvorenie DataFrame s výsledkami
results_data = {
    'Model': [
        '1. Jednoduchá Lineárna Regresia',
        '2. Viacnásobná Lineárna Regresia',
        '3. Rozšírená Lineárna Regresia',
        '4. Random Forest Regressor'
    ],
    'R-squared (R²)': [r2_simple, r2_multi, r2_expanded, r2_rf],
    'MAE (USD)': [mae_simple, mae_multi, mae_expanded, mae_rf],
    'RMSE (USD)': [rmse_simple, rmse_multi, rmse_expanded, rmse_rf]
}
results_df = pd.DataFrame(results_data)

print("--- Celkové Porovnanie Výkonnosti Modelov ---")
display(results_df.style.format({
    'R-squared (R²)': '{:.4f}',
    'MAE (USD)': '{:,.2f}',
    'RMSE (USD)': '{:,.2f}'
}).hide(axis="index"))

# Vizualizácia R-squared
plt.figure(figsize=(10, 6))
sns.barplot(x='R-squared (R²)', y='Model', data=results_df.sort_values('R-squared (R²)', ascending=False), palette='viridis')
plt.title('Porovnanie R-squared (R²) Skóre Modelov', fontsize=16)
plt.xlabel('R-squared (R²)', fontsize=12)
plt.ylabel('Model', fontsize=12)
plt.xlim(0, 1)
plt.tight_layout()
plt.show()

print("\nZáver porovnania: Random Forest Regressor dosiahol najvyššiu presnosť (R²), a preto je zvolený ako najlepší model.")

## 9. Interaktívny Prototyp s Gradio
Pre najlepší model (`RandomForestRegressor`) vytvoríme jednoduchú webovú aplikáciu pomocou knižnice Gradio. To umožní komukoľvek jednoducho získať odhad platu zadaním potrebných parametrov.

In [ ]:
import gradio as gr
import shap
import matplotlib.pyplot as plt
import io
from PIL import Image
import pandas as pd
import numpy as np

# Predpokladáme, že model_rf, X_expanded_encoded a explainer sú už definované

unique_experience = sorted(df['experience_level'].unique())
unique_company_size = sorted(df['company_size'].unique())
unique_employment_type = sorted(df['employment_type'].unique())
unique_job_title = sorted(df['job_title'].unique())
unique_company_location = sorted(df['company_location'].unique())

explainer = shap.TreeExplainer(model_rf)

def predict_salary_with_shap(experience, size, remote, emp_type, title, location):
    input_df = pd.DataFrame(columns=X_expanded_encoded.columns)
    input_df.loc[0] = 0

    input_df['remote_ratio'] = remote
    if f'experience_level_{experience}' in input_df.columns:
        input_df[f'experience_level_{experience}'] = 1
    if f'company_size_{size}' in input_df.columns:
        input_df[f'company_size_{size}'] = 1
    if f'employment_type_{emp_type}' in input_df.columns:
        input_df[f'employment_type_{emp_type}'] = 1
    if f'job_title_{title}' in input_df.columns:
        input_df[f'job_title_{title}'] = 1
    if f'company_location_{location}' in input_df.columns:
        input_df[f'company_location_{location}'] = 1

    prediction = model_rf.predict(input_df)[0]

    shap_values = explainer.shap_values(input_df)
    shap_vals = shap_values[0]
    feature_names = input_df.columns

    # Vyber top 10 podľa absolútnej hodnoty SHAP
    abs_shap_vals = np.abs(shap_vals)
    top_indices = abs_shap_vals.argsort()[-10:][::-1]
    top_features = feature_names[top_indices]
    top_shap_vals = shap_vals[top_indices]

    plt.figure(figsize=(8,6))
    plt.bar(top_features, top_shap_vals)
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("SHAP value (dopad na predikciu)")
    plt.title("Top 10 faktorov ovplyvňujúcich predikciu platu")
    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)
    img = Image.open(buf)

    return f"Predpokladaný ročný plat: ${prediction:,.2f} USD", img

iface = gr.Interface(
    fn=predict_salary_with_shap,
    inputs=[
        gr.Dropdown(unique_experience, label="Úroveň skúseností (Experience Level)"),
        gr.Dropdown(unique_company_size, label="Veľkosť firmy (Company Size)"),
        gr.Slider(0, 100, step=10, label="Práca na diaľku (% Remote)"),
        gr.Dropdown(unique_employment_type, label="Typ úväzku (Employment Type)"),
        gr.Dropdown(unique_job_title, label="Pracovná pozícia (Job Title)"),
        gr.Dropdown(unique_company_location, label="Lokalita firmy (Company Location)")
    ],
    outputs=[gr.Textbox(label="Odhadovaný plat"), gr.Image(type="pil", label="Interpretácia SHAP")],
    title="Prediktor Platov v AI Sektore s What-If analýzou",
    description="Zmeňte parametre a okamžite uvidíte predikovaný plat a vplyv jednotlivých faktorov.",
    allow_flagging='never'
)

iface.launch()


##  10 Zhrnutie a Záver

### Prehľad Vytvorených Modelov
V tomto projekte sme postupne vytvorili a vyhodnotili štyri regresné modely na predikciu platov v AI sektore:
1.  **Jednoduchá lineárna regresia:** Použila iba jednu premennú (`remote_ratio`) a ukázala sa ako nedostatočná (R² ≈ 0.0).
2.  **Viacnásobná lineárna regresia:** Pridaním `experience_level`, `company_size` a `employment_type` sa výkon výrazne zlepšil (R² ≈ 0.64).
3.  **Rozšírená viacnásobná lineárna regresia:** Zahrnutím `job_title` a `company_location` sa presnosť opäť dramaticky zvýšila (R² ≈ 0.85).
4.  **Random Forest Regressor:** Tento pokročilejší model, trénovaný na rovnakej rozšírenej sade príznakov, dosiahol najlepšie výsledky.

### Porovnanie a Výber Najlepšieho Modelu
Na základe metrík výkonnosti (MAE, RMSE a R-squared) bol **Random Forest Regressor** jednoznačne identifikovaný ako najlepší model. S hodnotou **R² 0.8537** a najnižšou priemernou chybou (MAE: **16,142 USD**) poskytol najpresnejšie a najspoľahlivejšie predikcie.

### Prototyp s Gradio
Ako finálny krok sme vytvorili interaktívny prototyp pomocou knižnice **Gradio**. Táto webová aplikácia umožňuje používateľom zadať kľúčové atribúty pracovnej pozície (úroveň skúseností, veľkosť firmy, lokalita atď.) a okamžite získať odhadovaný ročný plat na základe nášho najlepšieho modelu (Random Forest).

## 11. Zhrnutie a Záverečné Odporúčania projektu

### Zhrnutie Projektu
V tomto projekte sme úspešne vytvorili a porovnali štyri regresné modely na predikciu platov. Zistili sme, že pridávanie relevantných príznakov dramaticky zvyšuje presnosť modelu. Najvýkonnejším sa ukázal **Random Forest Regressor**, ktorý dosiahol **R-squared 0.8537**, čo znamená, že dokáže vysvetliť približne 85.4% variability v platoch.

### Návrhy na Ďalšie Kroky
1.  **Optimalizácia Hyperparametrov:** Použiť techniky ako `GridSearchCV` alebo `RandomizedSearchCV` na nájdenie najlepších nastavení pre Random Forest, čím sa môže presnosť ešte mierne zvýšiť.
2.  **Pokročilejší Feature Engineering:** Spracovať stĺpec `required_skills` a extrahovať kľúčové technológie (napr. Python, TensorFlow, PyTorch) ako samostatné príznaky.
3.  **Vyskúšanie Iných Modelov:** Experimentovať s modelmi ako **XGBoost** alebo **LightGBM**, ktoré sú často ešte výkonnejšie pre tabuľkové dáta.
4.  **Nasadenie do Produkcie:** Pripraviť model na reálne nasadenie pomocou nástrojov ako **Docker** a **FastAPI** a hosťovať ho na cloudovej platforme.